<a href="https://colab.research.google.com/github/pallateja96/Alage-detectection-CNES/blob/main/data_collection_s3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# First, you need to install the GEE API
!pip install earthengine-api


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip install folium


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:


!earthengine authenticate 



To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=C8MltajpRzGpnmkdY_ttwCaOeIxvmlbNmPsG7757708&tc=myWsxua8yrSjUQPnXX0qHuc7y0ysF7iMLyS8-nBYUzg&cc=_4vwzmN0di5zWWcPDvae1j5BbCYo1PASlkIhBF5QQCM

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWgavdeQokK2T30IFl2FsEgpZCL20T0PMmpS6LbQkxWP8o50pr-r8RMQsrs

Successfully saved authorization token.


In [22]:
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [13]:
# Next, import the necessary modules
import ee

# Initialize the API
ee.Initialize()

# Define the collection of Sentinel-3 OLCI images
olci = ee.ImageCollection("COPERNICUS/S3/OLCI")

# Define a date range for the images
date_range = ee.DateRange("2021-01-01", "2021-01-31")

# Filter the collection to include only images within the date range
olci = olci.filterDate(date_range)

# Define a region of interest (ROI) in the form of a geometry
# You can use any geometry that you want here
roi = ee.Geometry.Rectangle([-180, -90, 180, 90])

# Filter the collection to include only images that intersect the ROI
olci = olci.filterBounds(roi)

# Sort the images by the acquisition date
olci = olci.sort("system:time_start", False)

# Get the first image in the collection
image = olci.first()

# Print the image's properties
print(image.getInfo())


{'type': 'Image', 'bands': [{'id': 'Oa01_radiance', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [4866, 4553], 'crs': 'EPSG:32656', 'crs_transform': [311.5673507728058, 0, -100023.9334924072, 0, -311.5673507728058, 5828688.120035572]}, {'id': 'Oa02_radiance', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [4866, 4553], 'crs': 'EPSG:32656', 'crs_transform': [311.5673507728058, 0, -100023.9334924072, 0, -311.5673507728058, 5828688.120035572]}, {'id': 'Oa03_radiance', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [4866, 4553], 'crs': 'EPSG:32656', 'crs_transform': [311.5673507728058, 0, -100023.9334924072, 0, -311.5673507728058, 5828688.120035572]}, {'id': 'Oa04_radiance', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [4866, 4553], 'crs': 'EPSG:32656', 'crs_transform': [311.5673507728058, 0, -100023.9334924072, 0, -311.5673507728058, 5828688.120035572]}, {'id': 'Oa05_radiance', 'dat

In [77]:
from IPython.display import Image
# Import the Earth Engine Python API
import ee
import folium

# Initialize the Earth Engine API
ee.Initialize()

# Search for Sentinel-3 OLCI images
collection = ee.ImageCollection("COPERNICUS/S3/OLCI")

# Define a region of interest (ROI) in the form of a geometry
# You can use any geometry that you want here
roi = ee.Geometry.Polygon(
        [[[-82.58194756284642, 41.98707092163007],
          [-82.56890129819799, 41.89079147022274],
          [-82.41097283140111, 41.894880554318824],
          [-82.42779564634252, 41.99625711175054]]]);

# Filter the images by date and location
# you have to define your own date and region of interest
filtered_collection = collection.filterDate('2022-01-10', '2022-01-15').filterBounds(roi)

# Get the first image in the filtered collection
image = filtered_collection.first()

# Select the Red, Green, and Blue bands
image = image.select(['Oa04_radiance', 'Oa03_radiance', 'Oa02_radiance'])

# Get the projection of the ROI
roi_projection = ee.Image(1).projection()

#Map = image.visualize(bands= ['Oa04_radiance', 'Oa03_radiance', 'Oa02_radiance'], min= 0, max= 0.3)

#display(Map)

vis_params = {'bands': ['Oa04_radiance', 'Oa03_radiance', 'Oa02_radiance'], 'min': 0, 'max': 255}


mapid = image.getMapId(vis_params)
Map = folium.Map()
folium.TileLayer(
    tiles = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}".format(**mapid),
    attr = "Google Earth Engine",
    overlay = True,
    name = "image"
).add_to(Map)
#Map
# Print the image's properties
print(image.getInfo())

{'type': 'Image', 'bands': [{'id': 'Oa04_radiance', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [4866, 4585], 'crs': 'EPSG:32618', 'crs_transform': [321.78450131582196, 0, -461938.7521360088, 0, -321.78450131582196, 5859271.680550871]}, {'id': 'Oa03_radiance', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [4866, 4585], 'crs': 'EPSG:32618', 'crs_transform': [321.78450131582196, 0, -461938.7521360088, 0, -321.78450131582196, 5859271.680550871]}, {'id': 'Oa02_radiance', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [4866, 4585], 'crs': 'EPSG:32618', 'crs_transform': [321.78450131582196, 0, -461938.7521360088, 0, -321.78450131582196, 5859271.680550871]}], 'id': 'COPERNICUS/S3/OLCI/S3A_20220110T152724_20220110T153024', 'version': 1642119622321608, 'properties': {'SNAP_Graph_Processing_Framework_GPF_vers': '8.0.3', 'processing_time': 1641928661000, 'invalidPixelsPercent': 4, 'SNAP_Raster_Operators_vers': '8.0.

In [78]:
import ee

from google.colab import auth
from googleapiclient.discovery import build

# Initialize the Earth Engine API
ee.Initialize()

# Authenticate and construct the Drive API client
auth.authenticate_user()
drive_service = build('drive', 'v3')

# Define the export parameters
export_parameters = {
    'driveFolder': 'EE_Exports',
    'fileNamePrefix': 'my_image3',
    'crs': 'EPSG:32618',
    'region': roi.getInfo()['coordinates']
}

# Export the image to Google Drive
task = ee.batch.Export.image.toDrive(image, **export_parameters)
task.start()

# Print the task status
print(f'Task status: {task.status()["state"]}')

Task status: READY
